In [2]:
# !pip install xgboost
# !pip install refractio[snowflake]
# !pip uninstall urllib3 -y
# !pip install urllib3==1.26.15
# !pip install fosforml


In [9]:
from refractio import get_dataframe
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import xgboost as xgb
xgb.set_config(verbosity=2)
from xgboost import XGBRegressor
from math import sqrt

import urllib3
urllib3.__version__

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


'1.26.18'

In [10]:
df = get_dataframe("CPG_INVENTORY_DATA")
print(df.shape)
df.head()

Reading dataframe from snowflake native connector
(59722, 23)


,ORDER_NUMBER,ORDER_DATE,SKU_ID,WAREHOUSE_ID,CUSTOMER_TYPE,ORDER_QUANTITY,UNIT_SALE_PRICE,REVENUE,VENDOR_NAME,CURRENT_INVENTORY_QUANTITY,...,AVG_LEAD_TIME_IN_DAYS,MAX_LEAD_TIME_IN_DAYS,UNIT_PRICE,SKU_NAME,INVENTORY_DATE,CITY,PROVINCE,COUNTRY,LATITUDE,LONGITUDE
0,SO - 019101,05-01-2021,1009AA,AXW291,Distributor,1220,28.76326,35091.17720,McKesson Corp,7210,...,30,48,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998
1,SO - 019266,09-01-2021,1009AA,AXW291,Distributor,170,28.76326,4889.75420,McKesson Corp,7210,...,30,48,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998
2,SO - 019436,13-01-2021,1009AA,AXW291,Wholesale,905,28.76326,26030.75030,McKesson Corp,7210,...,30,48,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998
3,SO - 019508,15-01-2021,1009AA,AXW291,Wholesale,242,28.76326,6960.70892,McKesson Corp,7210,...,30,48,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998
4,SO - 019812,22-01-2021,1009AA,AXW291,Wholesale,100,28.76326,2876.32600,McKesson Corp,7210,...,30,48,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998


In [11]:
df["month"] = df["ORDER_DATE"].progress_apply(lambda x: str(x).split("-")[1])
df["year"] = df["ORDER_DATE"].progress_apply(lambda x: str(x).split("-")[-1])

100%|██████████| 59722/59722 [00:00<00:00, 850741.15it/s]


In [12]:
df.head()

,ORDER_NUMBER,ORDER_DATE,SKU_ID,WAREHOUSE_ID,CUSTOMER_TYPE,ORDER_QUANTITY,UNIT_SALE_PRICE,REVENUE,VENDOR_NAME,CURRENT_INVENTORY_QUANTITY,...,UNIT_PRICE,SKU_NAME,INVENTORY_DATE,CITY,PROVINCE,COUNTRY,LATITUDE,LONGITUDE,month,year
0,SO - 019101,05-01-2021,1009AA,AXW291,Distributor,1220,28.76326,35091.17720,McKesson Corp,7210,...,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021
1,SO - 019266,09-01-2021,1009AA,AXW291,Distributor,170,28.76326,4889.75420,McKesson Corp,7210,...,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021
2,SO - 019436,13-01-2021,1009AA,AXW291,Wholesale,905,28.76326,26030.75030,McKesson Corp,7210,...,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021
3,SO - 019508,15-01-2021,1009AA,AXW291,Wholesale,242,28.76326,6960.70892,McKesson Corp,7210,...,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021
4,SO - 019812,22-01-2021,1009AA,AXW291,Wholesale,100,28.76326,2876.32600,McKesson Corp,7210,...,28.76,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021


In [13]:
monthly_data = df.groupby(["SKU_ID","WAREHOUSE_ID","year","month"])[["ORDER_QUANTITY"]].sum()
monthly_data = monthly_data.reset_index()

In [14]:
monthly_data.head()

,SKU_ID,WAREHOUSE_ID,year,month,ORDER_QUANTITY
0,1009AA,AXW291,2021,01,2810
1,1009AA,AXW291,2021,02,1854
2,1009AA,AXW291,2021,03,1551
3,1009AA,AXW291,2021,04,708
4,1009AA,AXW291,2021,05,6056


In [15]:
monthly_data[(monthly_data["SKU_ID"]=="3551CA") & (monthly_data["WAREHOUSE_ID"] == "GUT930")].head()

,SKU_ID,WAREHOUSE_ID,year,month,ORDER_QUANTITY
40536,3551CA,GUT930,2021,01,1699
40537,3551CA,GUT930,2021,02,990
40538,3551CA,GUT930,2021,03,1025
40539,3551CA,GUT930,2021,04,45
40540,3551CA,GUT930,2021,05,1254


In [16]:
window = 13
_, *new_data = [[] for i in range(window+5)]

In [17]:
skus = df["SKU_ID"].unique()
warehouse = df["WAREHOUSE_ID"].unique()

In [18]:
for sku in tqdm(skus):
    for wh in warehouse:
#         if sku == "3551CA" and wh == "GUT930":
        frame_ = monthly_data[(monthly_data["SKU_ID"]==sku) & (monthly_data["WAREHOUSE_ID"]==wh)][["ORDER_QUANTITY","year","month"]]
        frame_ = frame_.reset_index()
        frame = frame_["ORDER_QUANTITY"]

        for i in range(0,frame.shape[0]-window+1):
            temp_ = list(frame.iloc[i:i+window])
            months = list(frame_.iloc[i:i+window]["month"])
            years = list(frame_.iloc[i:i+window]["year"])

            for _, val in enumerate(temp_):
                new_data[_].append(val)
            new_data[_+1].append(sku)
            new_data[_+2].append(wh)
            new_data[_+3].append(months[-1])
            new_data[_+4].append(years[-1])
                

100%|██████████| 290/290 [00:11<00:00, 24.49it/s]


In [19]:
data = pd.DataFrame({str(i): new_data[i-1] for i in range(1, len(new_data)+1)})
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2810,1854,1551,708,6056,6046,740,337,693,1598,1331,3370,972,1009AA,AXW291,01,2022
1,1854,1551,708,6056,6046,740,337,693,1598,1331,3370,972,500,1009AA,AXW291,02,2022
2,1551,708,6056,6046,740,337,693,1598,1331,3370,972,500,710,1009AA,AXW291,03,2022
3,708,6056,6046,740,337,693,1598,1331,3370,972,500,710,2358,1009AA,AXW291,04,2022
4,6056,6046,740,337,693,1598,1331,3370,972,500,710,2358,325,1009AA,AXW291,05,2022


In [20]:
y = data[f"{window}"]
X = data[[str(i) for i in range(1, window)]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=25)

In [21]:
y.head()


0     972
1     500
2     710
3    2358
4     325
Name: 13, dtype: int64

In [22]:
model = XGBRegressor(n_estimators=1200, max_depth=8, subsample=0.8, colsample_bytree=0.8, booster="gblinear", eta=0.001,
                     eval_metric="rmse")
model.fit(X_train, y_train)

/packages/Python-3.8-Snowpark/012b6c48-0652-40b7-adcf-cab27e6d11d7/3.8/xgboost/core.py:160: UserWarning: [11:48:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.001, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1200,
             n_jobs=None, num_parallel_tree=None, ...)

In [23]:
pred = model.predict(X_test)

In [24]:
pred = pd.DataFrame(pred)
pred.columns = ["prediction"]
pred.head()

,prediction
0,277.577576
1,266.989044
2,337.453766
3,263.854401
4,263.289856


In [25]:
y_test = pd.DataFrame(y_test)
y_test["pred"] = list(pred["prediction"])
y_test["pred"] = y_test["pred"].apply(lambda x: 1 if x < 0 else x) 

In [26]:
# mean_absolute_percentage_error(y_test["7"], y_test["pred"])

sqrt(mean_squared_error(y_test[f"{window}"], y_test["pred"]))

1912.1889215659885

In [27]:
y_test.head()

,13,pred
27603,43,277.577576
24765,5,266.989044
18189,90,337.453766
22566,1,263.854401
23737,1,263.289856


In [28]:
data["pred"] = list(model.predict(data[[str(i) for i in range(1, window)]]))

In [29]:
data[(data["14"]=="3551CA") & (data["15"] == "GUT930")].head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,pred
27024,1699,990,1025,45,1254,5158,4000,758,2100,1000,2500,700,559,3551CA,GUT930,01,2022,1735.675293
27025,990,1025,45,1254,5158,4000,758,2100,1000,2500,700,559,500,3551CA,GUT930,02,2022,1488.572144
27026,1025,45,1254,5158,4000,758,2100,1000,2500,700,559,500,559,3551CA,GUT930,03,2022,1332.158936
27027,45,1254,5158,4000,758,2100,1000,2500,700,559,500,559,559,3551CA,GUT930,04,2022,1195.069702
27028,1254,5158,4000,758,2100,1000,2500,700,559,500,559,559,1030,3551CA,GUT930,05,2022,1102.191528


In [30]:
def get_pred(x):
    output = list(data[(data["14"] == x[0]) & (data["15"] == x[1]) & (data["17"] == x[2]) & (data["16"] == x[3])]["pred"])
    if len(output) == 0:
        return x[4]
    return int(output[0])

df["pred"] = df[["SKU_ID","WAREHOUSE_ID","year","month","CURRENT_INVENTORY_QUANTITY"]].progress_apply(get_pred, axis =1 )

100%|██████████| 59722/59722 [07:41<00:00, 129.39it/s]


In [31]:
df.to_csv("prepared_cpg_test_data1.csv", index=False)

In [32]:
df[(df["SKU_ID"]=="1009AA") & (df["WAREHOUSE_ID"] == "AXW291")].head(100)

,ORDER_NUMBER,ORDER_DATE,SKU_ID,WAREHOUSE_ID,CUSTOMER_TYPE,ORDER_QUANTITY,UNIT_SALE_PRICE,REVENUE,VENDOR_NAME,CURRENT_INVENTORY_QUANTITY,...,SKU_NAME,INVENTORY_DATE,CITY,PROVINCE,COUNTRY,LATITUDE,LONGITUDE,month,year,pred
0,SO - 019101,05-01-2021,1009AA,AXW291,Distributor,1220,28.76326,35091.17720,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021,7210
1,SO - 019266,09-01-2021,1009AA,AXW291,Distributor,170,28.76326,4889.75420,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021,7210
2,SO - 019436,13-01-2021,1009AA,AXW291,Wholesale,905,28.76326,26030.75030,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021,7210
3,SO - 019508,15-01-2021,1009AA,AXW291,Wholesale,242,28.76326,6960.70892,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021,7210
4,SO - 019812,22-01-2021,1009AA,AXW291,Wholesale,100,28.76326,2876.32600,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2021,7210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SO - 055628,01-11-2022,1009AA,AXW291,Distributor,253,28.76326,7277.10478,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,11,2022,1012
96,SO - 055629,01-12-2022,1009AA,AXW291,Distributor,253,28.76326,7277.10478,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,12,2022,903
97,SO - 050461,15-01-2023,1009AA,AXW291,Wholesale,61,28.76326,1754.55886,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,01,2023,743
98,SO - 055630,01-02-2023,1009AA,AXW291,Distributor,190,28.76326,5465.01940,McKesson Corp,7210,...,Product 001,11-05-2023,Stanstead,Quebec,Canada,45.016666,-72.099998,02,2023,670


In [33]:
df[["SKU_ID","WAREHOUSE_ID","year","month", "ORDER_QUANTITY","CURRENT_INVENTORY_QUANTITY", "pred"]].head()

,SKU_ID,WAREHOUSE_ID,year,month,ORDER_QUANTITY,CURRENT_INVENTORY_QUANTITY,pred
0,1009AA,AXW291,2021,01,1220,7210,7210
1,1009AA,AXW291,2021,01,170,7210,7210
2,1009AA,AXW291,2021,01,905,7210,7210
3,1009AA,AXW291,2021,01,242,7210,7210
4,1009AA,AXW291,2021,01,100,7210,7210


In [41]:
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import configparser
from snowflake.snowpark.session import Session

config = configparser.ConfigParser()
config.read("snowflake_config.ini")

connection_params = {
    'user': f'{config["Snowflake"]["user"]}',
    'password': f'{config["Snowflake"]["password"]}',
    'account': f'{config["Snowflake"]["account"]}',
    'warehouse': f'{config["Snowflake"]["warehouse"]}',
    'database': f'{config["Snowflake"]["database"]}',
    'schema': f'{config["Snowflake"]["schema"]}',
    'role': f'{config["Snowflake"]["role"]}'
}
#print('connection_successful')
session = Session.builder.configs(connection_params).create()

## build a session with snowflake using above code

## use below code to first convert your pandas dataframe to a 
## snowflake df and write it to snowflake
df_snowflake=session.createDataFrame(
        df["pred"].values.tolist(),
        schema=df["pred"].columns.tolist())

df_snowflake.write.mode("overwrite").save_as_table("CPG_INVENTORY_DATA.FINAL_DATA.PREDICTED_INVENTORY_DATA")
## in the save_as_table give the database.schema.table name that you want
## to create in snowflake.

AttributeError: 'Series' object has no attribute 'column'

In [2]:
pip install fosforml

     |████████████████████████████████| 51kB 4.9MB/s eta 0:00:011
     |████████████████████████████████| 11.1MB 16.9MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 72.6MB/s eta 0:00:01
     |████████████████████████████████| 307kB 74.0MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 75.6MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 77.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 74.3MB/s eta 0:00:01
     |████████████████████████████████| 98.2MB 71.5MB/s eta 0:00:01     |██████████████▏                 | 43.3MB 71.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 80.5MB/s eta 0:00:01
     |████████████████████████████████| 174kB 77.3MB/s eta 0:00:01
     |████████████████████████████████| 40.0MB 62.1MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 73.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 43.0MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 56.8MB/s et

In [3]:
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

ModuleNotFoundError: No module named 'fosforml'

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    prediction = str(model.predict(data)[0])
    return prediction

In [27]:
payload = X_test.iloc[0].to_dict()

In [96]:
req = requests.Request()
req.json = {"payload":payload}
y = req
yo = score(model, y)
yo

'272.35406'

In [99]:
req.json

{'payload': {'1': 6,
  '2': 17,
  '3': 6,
  '4': 6,
  '5': 28,
  '6': 3,
  '7': 4,
  '8': 7,
  '9': 4,
  '10': 6,
  '11': 11,
  '12': 3}}

In [98]:
model_reg = register_model(model,
               score, 
               name="Demand_forecasting", 
               description="Forecasting_the_demand_of_the_inventory",
               flavour=MLModelFlavours.xgboost,
               model_type="regression",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install snowflake-connector-python[pandas] \\n pip install xgboost",
               y_true=y_test,
               y_pred=pred,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
